# VAE model training and selection

This notebook shows how the parameter space for B-VAEs can be explored for different values of the Beta and capacitance parameters. The theory is described at length in the author's [PhD](https://discovery.ucl.ac.uk/id/eprint/10134012/) and the accompanying preprint [Untangling urban data signatures](https://arxiv.org/abs/2106.15363).

See the accompanying [vae_exploration]() notebook showing how the selected VAE model is subsequently used.

## Notebook Setup

In [1]:
import os
from pathlib import Path

print(f"Launch directory: {os.getcwd()}")

if os.getcwd().endswith("/demos/vae"):
    os.chdir("..")
elif not os.getcwd().endswith("/demos"):
    print("Please run this notebook from the demos directory")
print(f"Working directory: {os.getcwd()}")

Launch directory: /Users/gareth/dev/benchmark-urbanism/cityseer-api/demos/vae
Working directory: /Users/gareth/dev/benchmark-urbanism/cityseer-api/demos


## Data preprocessing

A sample of the data is here provided for reproducibility of the methods. The sample dataset covers inner and north London and derives from a larger dataset which spans England and Wales.

Original versions of this work were applied to proprietary data (per the above PhD); however, this version of the dataset is purely based on open sources of data:

-  The road network is based on [OpenStreetMap](). This has been selected in this instance over [OS Open Roads](), which lacks pedestrian paths. The downloading, network cleaning, network decomposition, and network centrality calculations are done with the `cityseer` package.
- Information on eating and drinking establishments is derived from the [Food Standards Agency](). 

In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

from general_util import theme_setup

# load data
print((theme_setup.data_path / "df_north_london.feather").resolve())
df = pd.read_feather((theme_setup.data_path / "df_north_london.feather").resolve())
df = df.set_index("uid")
print(df.head())

# test indices
test_idxs = theme_setup.train_test_idxs(df, mod=200)

# preprocess data
X_raw, distances, labels = theme_setup.generate_theme(df, "all", bandwise=True, max_dist=None)
X_trans = StandardScaler().fit_transform(X_raw)

/Users/gareth/dev/benchmark-urbanism/cityseer-api/demos/data/df_north_london.feather
                             index  city_id              x              y  \
uid                                                                         
1-10934|90859±0±33463|10493      0        1  530342.873862  182325.087819   
1-330945707±5±357415633          1        1  527035.735672  187544.429602   
1-1459408967                     2        1  532164.776922  188193.929012   
1-26004973±6295862385            3        1  532001.390841  193891.914603   
1-7707806504                     4        1  535259.874968  180594.001376   

                             c_segment_density_100  c_segment_density_200  \
uid                                                                         
1-10934|90859±0±33463|10493             680.833374            2017.945068   
1-330945707±5±357415633                 216.558105             731.756958   
1-1459408967                            257.169739             809.

In [5]:
from IPython.utils import io
from vae.vae_util import vae_model, vae_runner

# setup params
epochs = 25
batch = 256
latent_dim = 6
lr = 1e-3
for seed in range(10):
    for beta in (0, 1, 2, 4, 8, 16, 32, 64):
        for cap in (0, 4, 8, 12, 16, 20):
            if beta == 0 and cap > 0:
                continue
            # generate identifying model key
            model_key = f"VAE_d{latent_dim}_b{beta}_c{cap}_s{seed}"
            dir_path = Path(theme_setup.weights_path / f"vae/seed_{seed}/{model_key}_e{epochs}_b{batch}_train")
            # skip if weights are already saved
            if dir_path.exists():
                print(f"Found existing weights, skipping {model_key}.")
                continue
            print(f"Processing {model_key}.")
            # creates model
            vae = vae_model.VAE(
                raw_dim=X_trans.shape[1],
                latent_dim=latent_dim,
                beta=beta,
                capacity=cap,
                epochs=epochs,
                model_key=model_key,
                seed=seed,
            )
            # trains model
            trainer = vae_runner.VAE_trainer(
                model=vae,
                X_samples=X_trans,
                labels=labels,
                distances=distances,
                epochs=epochs,
                batch=batch,
                best_loss=False,  # KL divergence increases with capacity
                lr=lr,
                save_path=dir_path,
                test_indices=test_idxs,
            )
            # reduce cell content output from training multiple models
            with io.capture_output() as captured:
                trainer.train()

INFO:vae.vae_util.vae_model:initialising VAE...
INFO:vae.vae_util.vae_runner:Epoch: 1


Found existing weights, skipping VAE_d6_b0_c0_s0.
Found existing weights, skipping VAE_d6_b1_c0_s0.
Found existing weights, skipping VAE_d6_b1_c4_s0.
Found existing weights, skipping VAE_d6_b1_c8_s0.
Found existing weights, skipping VAE_d6_b1_c12_s0.
Found existing weights, skipping VAE_d6_b1_c16_s0.
Found existing weights, skipping VAE_d6_b1_c20_s0.
Found existing weights, skipping VAE_d6_b2_c0_s0.
Found existing weights, skipping VAE_d6_b2_c4_s0.
Found existing weights, skipping VAE_d6_b2_c8_s0.
Found existing weights, skipping VAE_d6_b2_c12_s0.
Found existing weights, skipping VAE_d6_b2_c16_s0.
Found existing weights, skipping VAE_d6_b2_c20_s0.
Found existing weights, skipping VAE_d6_b4_c0_s0.
Found existing weights, skipping VAE_d6_b4_c4_s0.
Found existing weights, skipping VAE_d6_b4_c8_s0.
Found existing weights, skipping VAE_d6_b4_c12_s0.
Found existing weights, skipping VAE_d6_b4_c16_s0.
Found existing weights, skipping VAE_d6_b4_c20_s0.
Found existing weights, skipping VAE_d6_b

INFO:vae.vae_util.vae_model:updated capacity to 0.0
2022-04-29 11:21:32.115040: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
INFO:vae.vae_util.vae_runner:Step: 1: validation loss: 231.51303100585938
INFO:vae.vae_util.vae_runner:Epoch: 2
INFO:vae.vae_util.vae_model:updated capacity to 0.8333333134651184
INFO:vae.vae_util.vae_runner:Step: 2: validation loss: 197.88406372070312
INFO:vae.vae_util.vae_runner:Epoch: 3
INFO:vae.vae_util.vae_model:updated capacity to 1.6666666269302368
INFO:vae.vae_util.vae_runner:Step: 3: validation loss: 170.66226196289062
INFO:vae.vae_util.vae_runner:Epoch: 4
INFO:vae.vae_util.vae_model:updated capacity to 2.5
INFO:vae.vae_util.vae_runner:Step: 4: validation loss: 156.03561401367188
INFO:vae.vae_util.vae_runner:Epoch: 5
INFO:vae.vae_util.vae_model:updated capacity to 3.3333332538604736
INFO:vae.vae_util.vae_runner:Step: 5: validation loss: 151.46029663085938
INFO:vae.vae_util

Processing VAE_d6_b64_c0_s9.


INFO:vae.vae_util.vae_runner:Epoch: 1
INFO:vae.vae_util.vae_model:updated capacity to 0.0
INFO:vae.vae_util.vae_runner:Step: 1: validation loss: 268.5833435058594
INFO:vae.vae_util.vae_runner:Epoch: 2
INFO:vae.vae_util.vae_model:updated capacity to 0.0
INFO:vae.vae_util.vae_runner:Step: 2: validation loss: 259.0763244628906
INFO:vae.vae_util.vae_runner:Epoch: 3
INFO:vae.vae_util.vae_model:updated capacity to 0.0
INFO:vae.vae_util.vae_runner:Step: 3: validation loss: 258.4208984375
INFO:vae.vae_util.vae_runner:Epoch: 4
INFO:vae.vae_util.vae_model:updated capacity to 0.0
INFO:vae.vae_util.vae_runner:Step: 4: validation loss: 258.8081359863281
INFO:vae.vae_util.vae_runner:Epoch: 5
INFO:vae.vae_util.vae_model:updated capacity to 0.0
INFO:vae.vae_util.vae_runner:Step: 5: validation loss: 258.3542175292969
INFO:vae.vae_util.vae_runner:Epoch: 6
INFO:vae.vae_util.vae_model:updated capacity to 0.0
INFO:vae.vae_util.vae_runner:Step: 6: validation loss: 258.52313232421875
INFO:vae.vae_util.vae_ru

Found existing weights, skipping VAE_d6_b64_c4_s9.
Found existing weights, skipping VAE_d6_b64_c8_s9.
Found existing weights, skipping VAE_d6_b64_c12_s9.
Found existing weights, skipping VAE_d6_b64_c16_s9.
Found existing weights, skipping VAE_d6_b64_c20_s9.


In [6]:
from vae.vae_util import methods

methods.process_latents_vae()

Generating bandwise data


INFO:vae.vae_util.vae_model:initialising VAE...


...collecting seeds from models for beta 0 and capacitance [0]


INFO:vae.vae_util.vae_model:initialising VAE...
INFO:vae.vae_util.vae_model:initialising VAE...
INFO:vae.vae_util.vae_model:initialising VAE...
INFO:vae.vae_util.vae_model:initialising VAE...
INFO:vae.vae_util.vae_model:initialising VAE...
INFO:vae.vae_util.vae_model:initialising VAE...
INFO:vae.vae_util.vae_model:initialising VAE...
INFO:vae.vae_util.vae_model:initialising VAE...
INFO:vae.vae_util.vae_model:initialising VAE...
INFO:vae.vae_util.vae_model:initialising VAE...


...collecting seeds from models for beta 1 and capacitance [0, 4, 8, 12, 16, 20]


...

...collecting seeds from models for beta 64 and capacitance [0, 4, 8, 12, 16, 20]


INFO:vae.vae_util.vae_model:initialising VAE...
INFO:vae.vae_util.vae_model:initialising VAE...
INFO:vae.vae_util.vae_model:initialising VAE...
INFO:vae.vae_util.vae_model:initialising VAE...
INFO:vae.vae_util.vae_model:initialising VAE...
INFO:vae.vae_util.vae_model:initialising VAE...
INFO:vae.vae_util.vae_model:initialising VAE...
INFO:vae.vae_util.vae_model:initialising VAE...
INFO:vae.vae_util.vae_model:initialising VAE...
INFO:vae.vae_util.vae_model:initialising VAE...


...collecting seeds from models for beta 64 and capacitance [0, 4, 8, 12, 16, 20]


INFO:vae.vae_util.vae_model:initialising VAE...
INFO:vae.vae_util.vae_model:initialising VAE...
INFO:vae.vae_util.vae_model:initialising VAE...
INFO:vae.vae_util.vae_model:initialising VAE...
INFO:vae.vae_util.vae_model:initialising VAE...
INFO:vae.vae_util.vae_model:initialising VAE...
INFO:vae.vae_util.vae_model:initialising VAE...
INFO:vae.vae_util.vae_model:initialising VAE...
INFO:vae.vae_util.vae_model:initialising VAE...


NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for /Users/gareth/dev/benchmark-urbanism/cityseer-api/demos/temp/weights/vae/seed_9/VAE_d6_b64_c8_s9_e25_b256_train/weights

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


"""
Fine grid plots:
Each block is beta x capacity
"""
epochs = 25
seeds = list(range(10))
latent_dims = [6]
history_data = methods.gather_loss(seeds, epochs, latent_dims)
# returns 3d array - beta x caps x seeds
# load files if already calculated, otherwise compute, then save
udr_path = Path(theme_setup.weights_path / f"vae/data/udr_scores.npy")
udr_mask_path = Path(theme_setup.weights_path / f"vae/data/udr_mask_counts.npy")
if not udr_path.exists():
    print("UDR data does not exist, creating:")
    udr_arr, udr_mask_count_arr = methods.generate_udr_grid(latent_dims[0], epochs, seeds, kl_threshold=0.01)
    np.save(str(udr_path), udr_arr)
    np.save(str(udr_mask_path), udr_mask_count_arr)
else:
    print("Loading precomputed UDR data:")
    udr_arr = np.load(str(udr_path))
    udr_mask_count_arr = np.load(str(udr_mask_path))

In [ ]:
"""
VAE losses
"""
util_funcs.plt_setup()
fig, axes = plt.subplots(2, 4, figsize=(7, 5.5))
theme_sets = [
    ["val_loss", "val_rec_loss", "udr", "mask"],
    ["val_capacity_term", "val_kl", "val_kl_beta", "val_kl_beta_cap"],
]
for ax_row_n, (ax_row, theme_row) in enumerate(zip(axes, theme_sets)):
    for ax_col_n, (ax, theme) in enumerate(zip(ax_row, theme_row)):
        if theme == "udr":
            green_cmap = plt.get_cmap("Greens")
            # take mean across seeds on dimension 2
            udr_arr_mu = np.nanmean(udr_arr, axis=2)
            generate_arr_heatmap(ax, udr_arr_mu, cmap=green_cmap, constrain=(0, 1))
            ax.set_xlabel("UDR score")
        elif theme == "mask":
            green_cmap = plt.get_cmap("Greens")
            # take mean across seeds on dimension 2
            udr_mask_count_arr_mu = np.nanmean(udr_mask_count_arr, axis=2)
            generate_arr_heatmap(ax, udr_mask_count_arr_mu, cmap=green_cmap, constrain=(0, 1))
            ax.set_xlabel("Active axes")
        else:
            generate_heatmap(
                ax,
                theme,
                epochs,
                latent_dims[0],
                history_data,
                set_row_labels=ax_col_n == 0,
                set_col_labels=ax_row_n == 0,
            )
plt.suptitle(r"10 seed avg. losses for $\beta$ and $C$ hyperparameters (" + f"{epochs} epochs)")
path = f"model_scores_fine.pdf"
plt.savefig(plot_path / path, dpi=300)